In [15]:
#https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [16]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [17]:
def findCabinFloor(aDF):
    aDF.loc[aDF.Cabin.str[0] == 'A', 'Cabin'] = 1
    aDF.loc[aDF.Cabin.str[0] == 'B', 'Cabin'] = 2
    aDF.loc[aDF.Cabin.str[0] == 'C', 'Cabin'] = 3
    aDF.loc[aDF.Cabin.str[0] == 'D', 'Cabin'] = 4
    aDF.loc[aDF.Cabin.str[0] == 'E', 'Cabin'] = 5
    aDF.loc[aDF.Cabin.str[0] == 'F', 'Cabin'] = 6
    aDF.loc[aDF.Cabin.str[0] == 'G', 'Cabin'] = 7
    aDF.loc[aDF.Cabin.str[0] == 'T', 'Cabin'] = 8

In [19]:
# returns a DF that encodes the specified categorical variables.
from sklearn.preprocessing import LabelEncoder
def convertCat(aDFXVals,catCols):
    anotherDF = aDFXVals[catCols].apply(LabelEncoder().fit_transform)
    nonCatCols = aDFXVals.drop(catCols,axis=1)
    result = pd.concat([nonCatCols, anotherDF], axis=1)
    return result

In [18]:
def preprocessing(aDF):
    print("b4 processing: " + str(len(aDF)))
    #sklearn.preprocessing doesn't like nan
    aDF.Age.fillna(aDF['Age'].mode()[0], inplace=True)
    aDF.Cabin.fillna('0', inplace=True)
    aDF.Embarked.fillna(aDF['Embarked'].mode()[0],inplace=True)
    aDF.dropna(inplace = True)
    findCabinFloor(aDF)
    print("aftr processing: " + str(len(aDF)))
preprocessing(df)
preprocessing(test)

b4 processing: 891
aftr processing: 891
b4 processing: 418
aftr processing: 417


In [20]:
features= [c for c in df.columns.values if c  not in ['PassengerId','Survived','Cabin','Name','Ticket']]

newDF = convertCat(df[features],['Sex','Embarked'])

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
my_pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
# the scoring function defaults to the one in the random forest classifier
scores = cross_val_score(my_pipeline, newDF, df["Survived"])
print(scores)
print('Mean Absolute Error %2f' %(1-scores.mean()))

[ 0.77104377  0.81481481  0.79124579]
Mean Absolute Error 0.207632


In [23]:
print("b4 processing: " + str(len(test)))
#sklearn.preprocessing doesn't like nan
test = test.fillna(test['Age'].mode()[0])
#this numberic value screwes things up. There are 2 rows that have this value
test = test[test["Embarked"] != 24.0]
test = test.dropna()
print("aftr processing: " + str(len(test)))

b4 processing: 417
aftr processing: 417


In [24]:
testDF = convertCat(test[features],['Sex','Embarked'])
my_pipeline.fit(newDF,df["Survived"])
predictions = my_pipeline.predict(testDF)

In [25]:
my_submission = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
# you could use any filename. We choose submission here
#my_submission.to_csv('submission_pipelines1.csv', index=False)

In [13]:
#todo
#make a new feature that describes the deck of the person